In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [2]:
df = pd.read_csv("/content/Insurancepredc.csv")

In [3]:
df.head(15)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
5,31,female,25.740,0,no,southeast,3756.62160
6,46,female,33.440,1,no,southeast,8240.58960
7,37,female,27.740,3,no,northwest,7281.50560
8,37,male,29.830,2,no,northeast,6406.41070
9,60,female,25.840,0,no,northwest,28923.13692


In [5]:
df = df[["sex", "smoker", "region", "age", "bmi",'children','charges']].dropna()

In [7]:
le = LabelEncoder()
df["sex"] = le.fit_transform(df["sex"])
le1 = LabelEncoder()
df["smoker"] = le1.fit_transform(df["smoker"])
le2 = LabelEncoder()
df["region"] = le2.fit_transform(df["region"])

In [8]:
np.random.seed(42)
mask = np.random.rand(len(df)) < 0.5
df.loc[mask, "bmi"] = -1

In [17]:
X = df.drop(["bmi","smoker"],axis=1)
y = df["smoker"]

In [19]:
base_model = LogisticRegression()

In [20]:
self_training_model = SelfTrainingClassifier(base_model, criterion='k_best')

In [21]:
self_training_model.fit(X, y)

/usr/local/lib/python3.11/dist-packages/sklearn/semi_supervised/_self_training.py:288: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/semi_supervised/_self_training.py:293: UserWarning: k_best is larger than the amount of unlabeled samples. All unlabeled samples will be labeled in the first iteration
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SelfTrainingClassifier(criterion='k_best', estimator=LogisticRegression())

In [22]:
df["Predicted_Label"] = self_training_model.predict(X)

In [23]:
print(df.head(15))

    sex  smoker  region  age     bmi  children      charges  Predicted_Label
0     0       1       3   19  -1.000         0  16884.92400                1
1     1       0       2   18  33.770         1   1725.55230                0
2     1       0       2   28  33.000         3   4449.46200                0
3     1       0       1   33  22.705         0  21984.47061                1
4     1       0       1   32  -1.000         0   3866.85520                0
5     0       0       2   31  -1.000         0   3756.62160                0
6     0       0       2   46  -1.000         1   8240.58960                0
7     0       0       1   37  27.740         3   7281.50560                0
8     1       0       0   37  29.830         2   6406.41070                0
9     0       0       1   60  25.840         0  28923.13692                1
10    1       0       0   25  -1.000         0   2721.32080                0
11    0       1       2   62  26.290         0  27808.72510                1